# LSx Univariate

## Packages + Imports

In [ ]:
from nbdev.showdoc import *

from __future__ import annotations
from fastcore.docments import *
from fastcore.test import *
from fastcore.utils import *

import pandas as pd
import numpy as np
import itertools
from collections import defaultdict, Counter, deque
import warnings
import copy

from scipy import sparse
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator
from sklearn.exceptions import NotFittedError
import faiss

from dddex.baseClasses import BaseLSx, BaseWeightsBasedEstimator
from dddex.utils import restructureWeightsDataList

## Level-Set Approach based on DRF

The LSx approach based on Distributional Random Forest (DRF) has been dropped due to the drf package causing problems for new users during the installation of our dddex package.

In [ ]:
# from drf import drf 

# class LevelSetKDEx_DRF(BaseWeightsBasedEstimator, BaseLSx):
#     """
#     `LevelSetKDEx` turns any point forecasting model into an estimator of the underlying conditional density.
#     The name 'LevelSet' stems from the fact that this approach interprets the values of the point forecasts
#     as a similarity measure between samples. The point forecasts of the training samples are sorted and 
#     recursively assigned to a bin until the size of the current bin reaches `binSize` many samples. Then
#     a new bin is created and so on. For a new test sample we check into which bin its point prediction
#     would have fallen and interpret the training samples of that bin as the empirical distribution function
#     of this test sample.    
#     """
    
#     def __init__(self, 
#                  estimator, # Model with a .fit and .predict-method (implementing the scikit-learn estimator interface).
#                  binSize: int=100, # Size of the bins created while running fit.
#                  ):
        
#         super(BaseEstimator, self).__init__(estimator = estimator)

#         # Check if binSize is integer
#         if not isinstance(binSize, (int, np.int32, np.int64)):
#             raise ValueError("'binSize' must be an integer!")

#         self.binSize = binSize
        
#         self.yTrain = None
#         self.yPredTrain = None
#         self.drf = None
#         self.fitted = False
    
#     #---
    
#     def fit(self: LevelSetKDEx_DRF, 
#             X: np.ndarray, # Feature matrix used by `estimator` to predict `y`.
#             y: np.ndarray, # 1-dimensional target variable corresponding to the feature matrix `X`.
#             ):
#         """
#         Fit `LevelSetKDEx` model by grouping the point predictions of the samples specified via `X`
#         according to their value. Samples are recursively sorted into bins until each bin contains
#         `binSize` many samples. For details, checkout the function `generateBins` which does the
#         heavy lifting.
#         """
        
#         # Checks
#         if not isinstance(self.binSize, (int, np.int32, np.int64)):
#             raise ValueError("'binSize' must be an integer!")
            
#         if self.binSize > y.shape[0]:
#             raise ValueError("'binSize' mustn't be bigger than the size of 'y'!")
        
#         if X.shape[0] != y.shape[0]:
#             raise ValueError("'X' and 'y' must contain the same number of samples!")
        
#         #---
        
#         try:
#             yPred = self.estimator.predict(X)
            
#         except NotFittedError:
#             try:
#                 self.estimator.fit(X = X, y = y)                
#             except:
#                 raise ValueError("Couldn't fit 'estimator' with user specified 'X' and 'y'!")
#             else:
#                 yPred = self.estimator.predict(X)
        
#         #---
        
#         yPred = pd.DataFrame(yPred)
#         y = pd.Series(y)

#         DRF = drf(min_node_size = self.binSize, num_trees = 100, num_features = 1, honesty = False, sample_fraction = 0.5, response_scaling = False, mtry = 1, num_threads = 16)
#         DRF.fit(X = yPred, Y = y)
        
#         #---
        
#         # IMPORTANT: In case 'y' is given as a pandas.Series, we can potentially run into indexing 
#         # problems later on.
#         self.yTrain = y.ravel()
        
#         self.yPredTrain = yPred
#         self.drf = DRF
#         self.fitted = True
        
#     #---
    
#     def getWeights(self: LevelSetKDEx_DRF, 
#                    X: np.ndarray, # Feature matrix for which conditional density estimates are computed.
#                    # Specifies structure of the returned density estimates. One of: 
#                    # 'all', 'onlyPositiveWeights', 'summarized', 'cumDistribution', 'cumDistributionSummarized'
#                    outputType: str='onlyPositiveWeights', 
#                    # Optional. List with length X.shape[0]. Values are multiplied to the estimated 
#                    # density of each sample for scaling purposes.
#                    scalingList: list=None, 
#                    ) -> list: # List whose elements are the conditional density estimates for the samples specified by `X`.
        
#         # __annotations__ = BaseWeightsBasedEstimator.getWeights.__annotations__
#         __doc__ = BaseWeightsBasedEstimator.getWeights.__doc__
        
#         if not self.fitted:
#             raise NotFittedError("This LevelSetKDEx instance is not fitted yet. Call 'fit' with "
#                                  "appropriate arguments before trying to compute weights.")
        
#         #---
        
#         yPred = self.estimator.predict(X)
#         yPred = pd.DataFrame(yPred)
        
#         weightsArray = self.drf.predict(yPred).weights
#         weightsList = list(weightsArray)
#         weightsDataList = [(weights[weights > 0], np.where(weights > 0)[0]) for weights in weightsList]

#         weightsDataList = restructureWeightsDataList(weightsDataList = weightsDataList, 
#                                                      outputType = outputType, 
#                                                      y = self.yTrain,
#                                                      scalingList = scalingList,
#                                                      equalWeights = True)
        
#         return weightsDataList
    
    

## Level-Set Approach based on Clustering

I simply can't see any need for a method like that

In [ ]:
class LevelSetKDEx_clustering(BaseWeightsBasedEstimator, BaseLSx):
    """
    `LevelSetKDEx` turns any point forecasting model into an estimator of the underlying conditional density.
    The name 'LevelSet' stems from the fact that this approach interprets the values of the point forecasts
    as a similarity measure between samples. The point forecasts of the training samples are sorted and 
    recursively assigned to a bin until the size of the current bin reaches `binSize` many samples. Then
    a new bin is created and so on. For a new test sample we check into which bin its point prediction
    would have fallen and interpret the training samples of that bin as the empirical distribution function
    of this test sample.    
    """
    
    def __init__(self, 
                 estimator, # Model with a .fit and .predict-method (implementing the scikit-learn estimator interface).
                 nClusters: int=10, # Number of clusters to form as well as number of centroids to generate.
                 ):
        
        super(BaseEstimator, self).__init__(estimator = estimator)

        # nClusters must either be of type int, np.int64 or np.int32
        if isinstance(nClusters, (np.int32, np.int64)):
            nClusters = int(nClusters)
        
        elif not isinstance(nClusters, (int, np.int32, np.int64)):
            raise ValueError("'nClusters' must be an integer!")
                
        self.nClusters = nClusters

        self.yTrain = None
        self.yPredTrain = None
        self.kmeans = None
        self.clusterDict = None
        self.clusterSizes = None
        self.fitted = False
    
    #---
    
    def fit(self: LevelSetKDEx, 
            X: np.ndarray, # Feature matrix used by `estimator` to predict `y`.
            y: np.ndarray, # 1-dimensional target variable corresponding to the feature matrix `X`.
            ):
        """
        Fit `LevelSetKDEx` model by grouping the point predictions of the samples specified via `X`
        according to their value. Samples are recursively sorted into bins until each bin contains
        `binSize` many samples. For details, checkout the function `generateBins` which does the
        heavy lifting.
        """
        
        # Checks
        if self.nClusters is None:
            raise ValueError("'nClusters' must be specified to fit the LSx estimator!")
        
        # nClusters must either be of type int, np.int64 or np.int32
        if isinstance(self.nClusters, (np.int32, np.int64)):
            self.nClusters = int(self.nClusters)
        
        elif not isinstance(self.nClusters, (int, np.int32, np.int64)):
            raise ValueError("'nClusters' must be an integer!")
        
        # Check if nClusters is positive
        if self.nClusters <= 0:
            raise ValueError("'nClusters' must be positive!")
        
        if X.shape[0] != y.shape[0]:
            raise ValueError("'X' and 'y' must contain the same number of samples!")
        
        # IMPORTANT: In case 'y' is given as a pandas.Series, we can potentially run into indexing 
        # problems later on.
        if isinstance(y, pd.Series):
            y = y.ravel()
        
        #---
        
        try:
            yPred = self.estimator.predict(X)
            
        except NotFittedError:
            try:
                self.estimator.fit(X = X, y = y)                
            except:
                raise ValueError("Couldn't fit 'estimator' with user specified 'X' and 'y'!")
            else:
                yPred = self.estimator.predict(X)
        
        #---
        
        # Reshape yPred to 2D array with shape (nSamples, 1) and convert to float32.
        yPredMod = yPred.reshape(-1, 1).astype(np.float32)
        
        kmeans = faiss.Kmeans(d = 1, k = self.nClusters)
        kmeans.train(yPredMod)
        # self.centers = kmeans.centroids
        
        clusterAssignments = kmeans.assign(yPredMod)[1]
        
        # Get the cluster labels for each sample in yPred as a dict with keys being the cluster labels
        # and values being the indices of the samples in yPred that belong to that cluster.
        # Create another dict with keys being the cluster labels and values being the size of each cluster.
        clusterDict = defaultdict(list)
        clusterSizes = defaultdict(int)

        for index, cluster in enumerate(clusterAssignments):
            clusterDict[cluster].append(index)
            clusterSizes[cluster] += 1
        
        clusterSizes = pd.Series(clusterSizes)
       
        self.yTrain = y
        self.yPredTrain = yPred
        self.kmeans = kmeans
        self.clusterDict = clusterDict
        self.clusterSizes = clusterSizes
        self.fitted = True
        
    #---
    
    def getWeights(self, 
                   X: np.ndarray, # Feature matrix for which conditional density estimates are computed.
                   # Specifies structure of the returned density estimates. One of: 
                   # 'all', 'onlyPositiveWeights', 'summarized', 'cumDistribution', 'cumDistributionSummarized'
                   outputType: str='onlyPositiveWeights', 
                   # Optional. List with length X.shape[0]. Values are multiplied to the estimated 
                   # density of each sample for scaling purposes.
                   scalingList: list=None, 
                   ) -> list: # List whose elements are the conditional density estimates for the samples specified by `X`.
        
        # __annotations__ = BaseWeightsBasedEstimator.getWeights.__annotations__
        __doc__ = BaseWeightsBasedEstimator.getWeights.__doc__
        
        if not self.fitted:
            raise NotFittedError("This LevelSetKDEx instance is not fitted yet. Call 'fit' with "
                                 "appropriate arguments before trying to compute weights.")
        
        #---
        
        yPred = self.estimator.predict(X)
        # Reshape yPred to 2D array with shape (nSamples, 1) and convert to float32.
        yPred = yPred.reshape(-1, 1).astype(np.float32)

        # Get cluster labels of yPred
        clusterLabels = self.kmeans.assign(yPred)[1]
        
        #---
        
        weightsDataList = [(np.repeat(1 / self.clusterSizes[cluster], self.clusterSizes[cluster]), np.array(self.clusterDict[cluster], dtype = 'uintc')) 
                            for cluster in clusterLabels]

        weightsDataList = restructureWeightsDataList(weightsDataList = weightsDataList, 
                                                     outputType = outputType, 
                                                     y = self.yTrain,
                                                     scalingList = scalingList,
                                                     equalWeights = True)
        
        return weightsDataList
    

# LSx Multivariate

## Packages + Imports

In [ ]:
from __future__ import annotations
from fastcore.docments import *
from fastcore.test import *
from fastcore.utils import *

import pandas as pd
import numpy as np
import faiss
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment
from scipy.spatial import KDTree
from sklearn.base import BaseEstimator
from sklearn.exceptions import NotFittedError
from sklearn.tree import DecisionTreeRegressor

from collections import defaultdict
from joblib import Parallel, delayed, dump, load
import copy
import warnings

from dddex.baseClasses import BaseLSx, BaseWeightsBasedEstimator_multivariate
from dddex.levelSetKDEx_univariate import generateBins
from dddex.wSAA import SampleAverageApproximation, RandomForestWSAA, RandomForestWSAA_LGBM
from dddex.utils import restructureWeightsDataList_multivariate

## Level-Set Approach based on Clusters

LevelSetKDEx_multivariate is an older version of the currently used partition-based LSx model for multivariate point predictors. Both approaches use faiss-kMeans to create a partition of the target space based on the point forecasts. The difference between the two approaches is how we merge clusters in the subsequent steps to ensure that each cluster is big enough. Here, we simply merge each cluster that is too small (here defined as clusterSize < binSize / 2) with the next cluster and generate a bigger one. In the newer version, we don't actually merge them, but instead only take the observations of the next cluster into account when computing the weights.

In [ ]:
class LevelSetKDEx_multivariate(BaseWeightsBasedEstimator_multivariate, BaseLSx):
    """
    `LevelSetKDEx` turns any point forecasting model into an estimator of the underlying conditional density.
    The name 'LevelSet' stems from the fact that this approach interprets the values of the point forecasts
    as a similarity measure between samples. The point forecasts of the training samples are sorted and 
    recursively assigned to a bin until the size of the current bin reaches `binSize` many samples. Then
    a new bin is created and so on. For a new test sample we check into which bin its point prediction
    would have fallen and interpret the training samples of that bin as the empirical distribution function
    of this test sample.    
    """
    
    def __init__(self, 
                 estimator, # Model with a .fit and .predict-method (implementing the scikit-learn estimator interface).
                 binSize: int=None, # Size of the bins created while running fit.
                 # Determines behaviour of method `getWeights`. If False, all weights receive the same  
                 # value. If True, the distance of the point forecasts is taking into account.
                 equalBins: bool=False,
                 ):
        
        super(BaseEstimator, self).__init__(estimator = estimator)
        
        # Check if binSize is int
        if not isinstance(binSize, int):
            raise ValueError("'binSize' must be an integer!")
        
        # Check if equalBins is bool
        if not isinstance(equalBins, bool):
            raise ValueError("'equalBins' must be a boolean!")
        
        self.equalBins = equalBins
        self.binSize = binSize
        
        self.yTrain = None
        self.yPredTrain = None
        self.indicesPerBin = None
        self.lowerBoundPerBin = None
        self.fitted = False
    
    #---
    
    def fit(self: LevelSetKDEx, 
            X: np.ndarray, # Feature matrix used by `estimator` to predict `y`.
            y: np.ndarray, # 1-dimensional target variable corresponding to the feature matrix `X`.
            ):
        """
        Fit `LevelSetKDEx` model by grouping the point predictions of the samples specified via `X`
        according to their value. Samples are recursively sorted into bins until each bin contains
        `binSize` many samples. For details, checkout the function `generateBins` which does the
        heavy lifting.
        """
        
        # Checks
        if self.binSize is None:
            raise ValueError("'binSize' must be specified to fit the LSx estimator!")
            
        if self.binSize > y.shape[0]:
            raise ValueError("'binSize' mustn't be bigger than the size of 'y'!")
        
        if X.shape[0] != y.shape[0]:
            raise ValueError("'X' and 'y' must contain the same number of samples!")
        
        # IMPORTANT: In case 'y' is given as a pandas.Series, we can potentially run into indexing 
        # problems later on.
        if isinstance(y, pd.Series):
            y = y.ravel()
        
        #---
        
        try:
            yPred = self.estimator.predict(X)
            
        except NotFittedError:
            try:
                self.estimator.fit(X = X, y = y)                
            except:
                raise ValueError("Couldn't fit 'estimator' with user specified 'X' and 'y'!")
            else:
                yPred = self.estimator.predict(X)
        
        #---
        
        if len(y.shape) == 1:
            y = y.reshape(-1, 1)
            yPred = yPred.reshape(-1, 1)
        
        #---
        
        # Compute desired number of clusters dependend on binSize and number of samples
        nClusters = int(np.ceil(yPred.shape[0] / self.binSize))

        # Modify yPred to be compatible with faiss
        yPredMod = yPred.astype(np.float32)
        
        # Train kmeans model based on the faiss library
        kmeans = faiss.Kmeans(d = yPredMod.shape[1], k = nClusters)
        kmeans.train(yPredMod)

        # Get cluster centers created by faiss. IMPORTANT NOTE: not all clusters are used! We will handle that further below.
        centersAll = kmeans.centroids
        
        # Compute the cluster assignment for each sample
        if self.equalBins:
            clusterAssignments = self._getEqualSizedClusters(y = yPredMod)            
        else:
            clusterAssignments = kmeans.assign(yPredMod)[1]
        
        # Based on the clusters and cluster assignments, we can now compute the indices belonging to each bin / cluster
        indicesPerBin = defaultdict(list)
        binSizes = defaultdict(int)

        for index, cluster in enumerate(clusterAssignments):
            indicesPerBin[cluster].append(index)
            binSizes[cluster] += 1

        #---

        clustersUsed = np.array(list(indicesPerBin.keys()))
        clustersOrdered = np.sort(clustersUsed)

        centers = centersAll[clustersOrdered]
        indicesPerBin = [indicesPerBin[cluster] for cluster in clustersOrdered]
        binSizes = np.array([binSizes[cluster] for cluster in clustersOrdered])

        #---

        # Merge clusters that are too small (i.e. contain less than binSize / 2 samples).
        # clustersTooSmall is the array of all clusters that are too small.
        threshold = self.binSize / 2
        binsTooSmall = np.where(binSizes < threshold)[0]
        
        if len(binsTooSmall) > 0:

            # remove all centers from centersOld that are part of clustersTooSmall
            centersNew = np.delete(centers, binsTooSmall, axis = 0)
            centersTooSmall = centers[binsTooSmall]
            centersNew_oldIndices = np.delete(np.arange(len(centers)), binsTooSmall)

            KDTreeNew = KDTree(centersNew)
            clustersToMerge = KDTreeNew.query(centersTooSmall)[1]

            for i, clusterToMerge in enumerate(clustersToMerge):
                indicesPerBin[centersNew_oldIndices[clusterToMerge]].extend(indicesPerBin[binsTooSmall[i]])

            # Remove the indices given by clustersTooSmall from indicesPerBin by deleting the list entry
            indicesPerBin = [np.array(indices) for binIndex, indices in enumerate(indicesPerBin) if binIndex not in binsTooSmall]
            binSizes = [len(indices) for indices in indicesPerBin]
            binSizes = pd.Series(binSizes)

            self.centers = centersNew
            self.binSizes = binSizes
            self.kmeans = KDTreeNew
        
        else:
            self.centers = centers
            self.binSizes = pd.Series(binSizes)
            self.kmeans = KDTree(self.centers)

            # Transform the indices given by indicesPerBin into numpy arrays
            indicesPerBin = [np.array(indices) for indices in indicesPerBin]
            
        #---
        
        self.yTrain = y
        self.yPredTrain = yPred
        self.indicesPerBin = indicesPerBin
        self.fitted = True
        
        
    #---
    
    def _getEqualSizedClusters(self,
                               y,
                               ):
            
        centers = self.centers.reshape(-1, 1, y.shape[-1]).repeat(self.binSize, 1).reshape(-1, y.shape[-1])

        distance_matrix = cdist(y, centers)
        clusterAssignments = linear_sum_assignment(distance_matrix)[1]//self.binSize

        return clusterAssignments

    #---
    
    def getWeights(self, 
                   X: np.ndarray, # Feature matrix for which conditional density estimates are computed.
                   # Specifies structure of the returned density estimates. One of: 
                   # 'all', 'onlyPositiveWeights', 'summarized', 'cumDistribution', 'cumDistributionSummarized'
                   outputType: str='onlyPositiveWeights', 
                   # Optional. List with length X.shape[0]. Values are multiplied to the estimated 
                   # density of each sample for scaling purposes.
                   scalingList: list=None, 
                   ) -> list: # List whose elements are the conditional density estimates for the samples specified by `X`.
        
        # __annotations__ = BaseWeightsBasedEstimator.getWeights.__annotations__
        __doc__ = BaseWeightsBasedEstimator_multivariate.getWeights.__doc__
        
        if not self.fitted:
            raise NotFittedError("This LevelSetKDEx instance is not fitted yet. Call 'fit' with "
                                 "appropriate arguments before trying to compute weights.")
        
        #---
        
        yPred = self.estimator.predict(X).astype(np.float32)
        
        if len(yPred.shape) == 1:
            yPred = yPred.reshape(-1, 1)
            
        #---
        
        if self.equalBins:
            binPerPred = self._getEqualSizedClusters(y = yPred)
            
        else:
            binPerPred = self.kmeans.query(yPred)[1]
        
        #---
        
        neighborsList = [self.indicesPerBin[binIndex] for binIndex in binPerPred]
        
        weightsDataList = [(np.repeat(1 / len(neighbors), len(neighbors)), np.array(neighbors)) for neighbors in neighborsList]
        
        weightsDataList = restructureWeightsDataList_multivariate(weightsDataList = weightsDataList, 
                                                                  outputType = outputType, 
                                                                  y = self.yTrain,
                                                                  scalingList = scalingList,
                                                                  equalWeights = True)
        
        return weightsDataList
    

# wSAA

## Packages + Imports

In [ ]:
from __future__ import annotations
from fastcore.docments import *
from fastcore.test import *
from fastcore.utils import *

import pandas as pd
import numpy as np
import copy
from collections import defaultdict

from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.base import MetaEstimatorMixin
from lightgbm.sklearn import LGBMModel
from dddex.baseClasses import BaseWeightsBasedEstimator
from dddex.utils import restructureWeightsDataList, restructureWeightsDataList_multivariate

## Random Forest Speed Up by Lookup Table

The weights calculation of single trees can be speed up massively by viewing the tree as a partition algorithm. As such, we can simply use a lookup table for the calculation of the weights once we know into which leaf-node a new observation falls. Here, we tried to apply this idea to Random Forest as well. Sadly, it turns out that the assembling of the weights of each single tree into a single set of weights is more computationally expensive than the old way of calculating the weights for a Random Forest model. Maybe we can come up with a solution in the future. It might be worth looking into the R code of the Distributional Random Forest package.

In [ ]:
# We attempt here to speed up the computation of the weights by interpreting every single
# tree as a lookup table. This way we don't have to compare the leaf-Indices arrays of each
# training sample and each test sample.
# Unfortunately, despite the fact that this strategy works very well for a single tree,
# it doesn't work for the whole forest because the structure of the output of the lookup 
# tables per tree makes it difficult to aggregate the received weights per tree 
# over all trees.

class RandomForestWSAA_speedup(RandomForestRegressor, BaseWeightsBasedEstimator):
    
    def fit(self, 
            X: np.ndarray, # Feature matrix
            y: np.ndarray, # Target values
            **kwargs):

        super().fit(X = X, 
                    y = y, 
                    **kwargs)
        
        self.yTrain = y
        
        leafIndices = self.apply(X)

        indicesPerBinPerTree = list()

        for indexTree in range(self.n_estimators):
            leafIndicesPerTree = leafIndices[:, indexTree]

            indicesPerBin = defaultdict(list)

            for index, leafIndex in enumerate(leafIndicesPerTree):
                indicesPerBin[leafIndex].append(index)

            indicesPerBinPerTree.append(indicesPerBin)
        
        self.indicesPerBinPerTree = indicesPerBinPerTree

        
    
    #---
    
    def getWeights(self, 
                   X: np.ndarray, # Feature matrix for which conditional density estimates are computed.
                   # Specifies structure of the returned density estimates. One of: 
                   # 'all', 'onlyPositiveWeights', 'summarized', 'cumDistribution', 'cumDistributionSummarized'
                   outputType: str='onlyPositiveWeights', 
                   # Optional. List with length X.shape[0]. Values are multiplied to the estimated 
                   # density of each sample for scaling purposes.
                   scalingList: list=None, 
                   ) -> list: # List whose elements are the conditional density estimates for the samples specified by `X`.
        
        __doc__ = BaseWeightsBasedEstimator.getWeights.__doc__
        
        #---
        
        leafIndicesPerTree = self.apply(X)
        
        weightsDataList = list()

        for leafIndices in leafIndicesPerTree:
            
            weights = np.zeros(self.yTrain.shape[0])

            for indexTree in range(len(leafIndices)):
                indicesPosWeight = self.indicesPerBinPerTree[indexTree][leafIndices[indexTree]]

                weightsNew = np.zeros(self.yTrain.shape[0])
                np.put(weightsNew, indicesPosWeight, 1 / len(indicesPosWeight))
                
                weights = weights + weightsNew

            weights = weights / len(leafIndices)

            weightsPosIndex = np.where(weights > 0)[0]

            weightsDataList.append((weights[weightsPosIndex], weightsPosIndex))

        #---

        # Check if self.yTrain is a 2D array with more than one column.
        if len(self.yTrain.shape) > 1:
            if self.yTrain.shape[1] > 1:

                if not outputType in ['all', 'onlyPositiveWeights', 'summarized']:
                    raise ValueError("outputType must be one of 'all', 'onlyPositiveWeights', 'summarized' for multivariate y.")
                
                weightsDataList = restructureWeightsDataList_multivariate(weightsDataList = weightsDataList, 
                                                                        outputType = outputType, 
                                                                        y = self.yTrain, 
                                                                        scalingList = scalingList,
                                                                        equalWeights = False) 
            
        else:
            weightsDataList = restructureWeightsDataList(weightsDataList = weightsDataList, 
                                                        outputType = outputType, 
                                                        y = self.yTrain, 
                                                        scalingList = scalingList,
                                                        equalWeights = False)
            
        
                

        return weightsDataList
    
    #---
    
    def predict(self: BaseWeightsBasedEstimator, 
                X: np.ndarray, # Feature matrix for which conditional quantiles are computed.
                probs: list, # Probabilities for which quantiles are computed.
                outputAsDf: bool=True, # Determines output. Either a dataframe with probs as columns or a dict with probs as keys.
                # Optional. List with length X.shape[0]. Values are multiplied to the predictions
                # of each sample to rescale values.
                scalingList: list=None, 
                ): 
        
        __doc__ = BaseWeightsBasedEstimator.predict.__doc__
        
        return super(MetaEstimatorMixin, self).predict(X = X,
                                                       probs = probs, 
                                                       scalingList = scalingList)
    
    #---
    
    def pointPredict(self,
                     X: np.ndarray, # Feature Matrix
                     **kwargs):
        """Original `predict` method to generate point forecasts"""
        
        return super().predict(X = X,
                               **kwargs)


# Cross Validation

## Packages + Imports

In [ ]:
from __future__ import annotations
from fastcore.docments import *
from fastcore.test import *
from fastcore.utils import *

import pandas as pd
import numpy as np

from joblib import Parallel, delayed, dump, load
from collections import defaultdict
import copy

from sklearn.model_selection import ParameterGrid, ParameterSampler
from sklearn.base import clone

from dddex.baseClasses import BaseLSx
from dddex.wSAA import SampleAverageApproximation

The whole idea of using the Wasserstein Distance between the validation points and the computed densities turned out to be mathematically bullshit. 

## CV - Wasserstein Distance

In [ ]:
class DensityCrossValidation:
    """
    Class to efficiently tune the `binSize` parameter of all Level-Set based models.
    """
    
    def __init__(self,
                 # An object with a `predict` method that must (!) have an argument called `probs`
                 # that specifies which quantiles to predict. Further, `estimator` needs
                 # a `set_params` and `fit` method.
                 estimator, 
                 cvFolds, # An iterable yielding (train, test) splits as arrays of indices.
                 # dict or list of dicts with parameters names (`str`) as keys and distributions
                 # or lists of parameters to try. Distributions must provide a ``rvs``
                 # method for sampling (such as those from scipy.stats.distributions).
                 parameterGrid: dict, 
                 randomSearch: bool=False, # Whether to use randomized search or grid search
                 # Number of parameter settings that are sampled if `randomSearch == True`. 
                 # n_iter trades off runtime vs quality of the solution.
                 nIter: int=None,
                 p: int=1, # The order of the wasserstein distance to evaluate each hyperparameter settings.
                 n_jobs: int=None, # Number of jobs to run in parallel.
                 # Pseudo random number generator state used for random uniform sampling of parameter candidate values.
                 # Pass an int for reproducible output across multiple function calls.
                 random_state: int=None, 
                 ):
        
        # CHECKS  
        # if not isinstance(estimatorLSx, (BaseLSx)):
        #     raise ValueError("'estimatorLSx' has to be a 'LevelSetKDEx', 'LevelSetKDEx_NN_new' or a 'LevelSetKDEx_kNN' object!")               
        
        if not isinstance(p, int):
            raise ValueError("`p` must be an integer between 1 and inf!")
            
        #---
        
        if randomSearch:
            self.parameterGrid = list(ParameterSampler(param_distributions = parameterGrid,
                                                       n_iter = nIter,
                                                       random_state = random_state).__iter__())
            
            self.randomSearch = True
            self.nIter = nIter
            self.random_state = random_state
            
        else:
            self.parameterGrid = ParameterGrid(parameterGrid)
            self.randomSearch = False
            
        #---
        
        self.estimator = copy.deepcopy(estimator)
        self.cvFolds = cvFolds
        self.p = p
        self.n_jobs = n_jobs
        
        self.bestParams = None
        self.bestEstimator = None
        self.cvResults = None
        self.cvResults_raw = None
        

In [ ]:
@patch
def fit(self: DensityCrossValidation, 
        X: np.ndarray, # Feature matrix (has to work with the folds specified via `cvFolds`)
        y: np.ndarray, # Target values (has to work with the folds specified via `cvFolds`)
        ): 
    
    # Making sure that X and y are arrays to ensure correct subsetting via implicit indices.
    X = np.array(X)
    y = np.array(y)
    
    # scoresPerFold = Parallel(n_jobs = self.n_jobs)(delayed(getFoldScore_wasserstein)(estimator = copy.deepcopy(self.estimator),
    #                                                                                  parameterGrid = self.parameterGrid,
    #                                                                                  cvFold = cvFold,
    #                                                                                  p = self.p,
    #                                                                                  X = X,
    #                                                                                  y = y) for cvFold in self.cvFolds)
    
    scoresPerFold = [getFoldScore_wasserstein(estimator = copy.deepcopy(self.estimator),
                                              parameterGrid = self.parameterGrid,
                                              cvFold = cvFold,
                                              p = self.p,
                                              y = y,
                                              X = X) for cvFold in self.cvFolds]
    
    # RESULTS
    self.cvResults_raw = scoresPerFold
    meanCostsDf = sum(scoresPerFold) / len(scoresPerFold)
    self.cvResults = meanCostsDf
    
    # BEST PARAMETER SETTING
    meanCostsPerParam = meanCostsDf.mean(axis = 1)
    paramsBest = meanCostsPerParam.index[np.argmin(meanCostsPerParam)]
    paramsBest = dict(zip(meanCostsPerParam.index.names, paramsBest))
    self.bestParams = paramsBest
    
    # REFITTING ESTIMATOR WITH BEST PARAMETERS
    estimator = copy.deepcopy(self.estimator)
    estimator.set_params(**paramsBest)
    estimator.fit(X = X, y = y)

    self.bestEstimator = estimator

### Scores for Single Fold

In [ ]:
# This function evaluates the newsvendor performance for different bin sizes for one specific fold.
# The considered bin sizes

def getFoldScore_wasserstein(estimator, parameterGrid, cvFold, p, X, y):
    
    indicesTrain = cvFold[0]
    indicesTest = cvFold[1]
    
    yTrainFold = y[indicesTrain]
    XTrainFold = X[indicesTrain]
    
    yTestFold = y[indicesTest]
    XTestFold = X[indicesTest]
    
    #---

    SAA_fold = SampleAverageApproximation()
    SAA_fold.fit(y = yTrainFold)

    densitiesSAA = SAA_fold.getWeights(X = XTestFold, outputType = 'onlyPositiveWeightsValues')
    
    wassersteinDistSAA = getWassersteinDistances(densities = densitiesSAA,
                                                 yTest = yTestFold,
                                                 p = p).sum()
    
    #---
    
    # Necessary to ensure compatability with wSAA-models etc.
    try:
        estimator.refitPointEstimator(X = XTrainFold, 
                                      y = yTrainFold)
    except:
        pass

    costsPerParam = defaultdict(dict)

    for params in parameterGrid:

        estimator.set_params(**params)

        estimator.fit(X = XTrainFold,
                      y = yTrainFold)
        
        #---
        
        densities = estimator.getWeights(X = XTestFold,
                                         outputType = 'onlyPositiveWeightsValues')
        
        wassersteinDist = getWassersteinDistances(densities = densities, 
                                                  yTest = yTestFold, 
                                                  p = p).sum()
        
        if wassersteinDistSAA > 0:
            wassersteinRatio = wassersteinDist / wassersteinDistSAA
        else:
            if wassersteinDist == 0:
                wassersteinRatio = 0
            else:
                wassersteinRatio = 1
                
        costsPerParam[tuple(params.values())] = {'wassersteinRatio': wassersteinRatio}

    #---
    # s = pd.Series(list(d.values()),index=pd.MultiIndex.from_tuples(d.keys()))
    costsDf = pd.DataFrame.from_dict(costsPerParam, orient = 'index')
    costsDf.index.names = list(params.keys())
    
    return costsDf

## CV - Combined - Wasserstein Distance

In [ ]:
class DensityCrossValidationLSx:
    """
    Class to efficiently tune the `binSize` parameter of all Level-Set based models.
    """
    
    def __init__(self,
                 estimatorLSx, # A Level-Set based model.
                 cvFolds, # An iterable yielding (train, test) splits as arrays of indices.
                 # dict or list of dicts with LSx parameters names (`str`) as keys and distributions
                 # or lists of parameters to try. Distributions must provide a ``rvs``
                 # method for sampling (such as those from scipy.stats.distributions).
                 parameterGridLSx: dict,
                 # dict or list of dicts with parameters names (`str`) of the point predictor as keys
                 # and distributions or lists of parameters to try. Distributions must provide a ``rvs``
                 # method for sampling (such as those from scipy.stats.distributions).
                 parameterGridEstimator: dict,
                 randomSearchLSx: bool=False, # Whether to use randomized search or grid search for the LSx parameters.
                 # Whether to use randomized search or grid search for the point predictor parameters.
                 randomSearchEstimator: bool=False, 
                 # Number of parameter settings of the LSx model that are sampled if `randomSearchLSx == True`. 
                 # LSx parameter settings are usually relatively cheap to evaluate, so all sampled LSx paramater settings
                 # are evaluated for each point predictor parameter setting.
                 nIterLSx: int=None,
                 # Number of parameter settings of the underlying point predictor that are sampled if 
                 # `randomSearchEstimator == True`. nIterEstimator trades off runtime vs quality of the solution.
                 nIterEstimator: int=None,
                 p: int=1, # The order of the wasserstein distance to evaluate each hyperparameter settings.
                 n_jobs: int=None, # number of folds being computed in parallel.
                 # Pseudo random number generator state used for random uniform sampling of parameter candidate values.
                 # Pass an int for reproducible output across multiple function calls.
                 random_state: int=None,
                 ):
        
        # CHECKS  
        if not isinstance(estimatorLSx, (BaseLSx)):
            raise ValueError("'estimatorLSx' has to be a 'LevelSetKDEx', 'LevelSetKDEx_NN_new' or a 'LevelSetKDEx_kNN' object!")
            
        if len(parameterGridEstimator) == 0:
            raise ValueError("No parameter candidates have been specified for the point predictor. If you want to only evaluate"
                             "parameter settings of the LSx-estimator itself, use the class `QuantileCrossValidation` instead or"
                             "provide a fixed parameter setting for the point predictor via `parameterGridEstimator`.")
            
        if len(parameterGridLSx) == 0:
            raise ValueError("No parameter candidates have been specified for the LSx model! If you want to only evaluate"
                             "parameter settings of the point predictor, use standard cross-validation classes or instead"
                             "provide a fixed parameter setting for the LS model via `parameterGridLSx`.")
            
        if not isinstance(p, int):
            raise ValueError("`p` must be an integer between 1 and inf!")
        
        #---
        
        if randomSearchLSx:
            self.parameterGridLSx = list(ParameterSampler(param_distributions = parameterGridLSx,
                                                          n_iter = nIterLSx,
                                                          random_state = random_state).__iter__())
            self.randomSearchLSx = True
            self.nIterLsx = nIterLSx
            self.random_state = random_state
        
        else:
            self.parameterGridLSx = ParameterGrid(parameterGridLSx)
            self.randomSearchLSx = False
            
        
        if randomSearchEstimator:
            
            self.parameterGridEstimator = list(ParameterSampler(param_distributions = parameterGridEstimator,
                                                                n_iter = nIterEstimator,
                                                                random_state = random_state).__iter__())
            self.randomSearchEstimator = True
            self.nIterEstimator = nIterEstimator
            self.random_state = random_state
            
        else:
            self.parameterGridEstimator = ParameterGrid(parameterGridEstimator)
            self.randomSearchEstimator = False
            
        #---
        
        self.estimatorLSx = copy.deepcopy(estimatorLSx)
        
        self.cvFolds = cvFolds
        self.p = p
        self.n_jobs = n_jobs
        
        self.bestParams = None
        self.bestEstimator = None
        self.cvResults = None
        self.cvResults_raw = None
        

In [ ]:
@patch
def fit(self: DensityCrossValidationLSx, 
        X: np.ndarray, # Feature matrix (has to work with the folds specified via `cvFolds`)
        y: np.ndarray, # Target values (has to work with the folds specified via `cvFolds`)
        ): 
    
    # Making sure that X and y are arrays to ensure correct subsetting via implicit indices.
    X = np.array(X)
    y = np.array(y)
    
    scoresPerFold = Parallel(n_jobs = self.n_jobs)(delayed(getFoldScoreLSx_wasserstein)(estimatorLSx = copy.deepcopy(self.estimatorLSx),
                                                                                        parameterGridLSx = self.parameterGridLSx, 
                                                                                        parameterGridEstimator = self.parameterGridEstimator,
                                                                                        cvFold = cvFold,
                                                                                        p = self.p,
                                                                                        y = y,
                                                                                        X = X) for cvFold in self.cvFolds)
    
    # scoresPerFold = [getFoldScoreLSx_wasserstein(estimatorLSx = copy.deepcopy(self.estimatorLSx),
    #                                              parameterGridLSx = self.parameterGridLSx,
    #                                              parameterGridEstimator = self.parameterGridEstimator,
    #                                              cvFold = cvFold,
    #                                              p = self.p,
    #                                              y = y,
    #                                              X = X) for cvFold in self.cvFolds]

    self.cvResults_raw = scoresPerFold
    meanCostsDf = sum(scoresPerFold) / len(scoresPerFold)
    self.cvResults = meanCostsDf
    
    #---
    
    # BEST PARAMETER SETTINGS OVER ALL PROBS
    meanCostsPerParam = meanCostsDf.mean(axis = 1)
    paramsBest = meanCostsPerParam.index[np.argmin(meanCostsPerParam)]
    paramsBest = dict(zip(meanCostsPerParam.index.names, paramsBest))
    
    paramsLSxNames = self.estimatorLSx._get_param_names()
    paramsLSxBest = {paramName: value for paramName, value in paramsBest.items() if paramName in paramsLSxNames}
    paramsEstimatorBest = {paramName: value for paramName, value in paramsBest.items() if not paramName in paramsLSxNames}
    
    self.bestParams = paramsBest
    self.bestParamsLSx = paramsLSxBest
    self.bestParamsEstimator = paramsEstimatorBest
        
    #---
    
    estimatorLSx = copy.deepcopy(self.estimatorLSx)
    
    estimator = clone(estimatorLSx.estimator)
    estimator.set_params(**paramsEstimatorBest)
    estimator.fit(X = X, y = y)
    
    estimatorLSx.set_params(**paramsLSxBest,
                            estimator = estimator)
    estimatorLSx.fit(X = X, y = y)

    self.bestEstimator = estimatorLSx

In [ ]:
# show_doc(CrossValidationLSx_combined.fit)

### Scores for Single Fold

In [ ]:
# This function evaluates the newsvendor performance for different bin sizes for one specific fold.
# The considered bin sizes

def getFoldScoreLSx_wasserstein(estimatorLSx, parameterGridLSx, parameterGridEstimator, cvFold, p, X, y):
    
    indicesTrain = cvFold[0]
    indicesTest = cvFold[1]
    
    yTrainFold = y[indicesTrain]
    XTrainFold = X[indicesTrain]
    
    yTestFold = y[indicesTest]
    XTestFold = X[indicesTest]
    
    #---

    SAA_fold = SampleAverageApproximation()
    SAA_fold.fit(y = yTrainFold)

    densitiesSAA = SAA_fold.getWeights(X = XTestFold, outputType = 'onlyPositiveWeightsValues')
    
    wassersteinDistSAA = getWassersteinDistances(densities = densitiesSAA,
                                                 yTest = yTestFold,
                                                 p = p).sum()
    
    #---
    
    costsDfList = list()
    
    for paramsEstimator in parameterGridEstimator:
        
        estimatorLSx.refitPointEstimator(X = XTrainFold, 
                                         y = yTrainFold,
                                         **paramsEstimator)

        #---

        costsPerParamLSx = defaultdict(dict)

        for paramsLSx in parameterGridLSx:

            estimatorLSx.set_params(**paramsLSx)

            estimatorLSx.fit(X = XTrainFold,
                             y = yTrainFold)

            densities = estimatorLSx.getWeights(X = XTestFold,
                                                outputType = 'onlyPositiveWeightsValues')
        
            wassersteinDist = getWassersteinDistances(densities = densities, 
                                                      yTest = yTestFold, 
                                                      p = p).sum()

            if wassersteinDistSAA > 0:
                wassersteinRatio = wassersteinDist / wassersteinDistSAA
            else:
                if wassersteinDist == 0:
                    wassersteinRatio = 0
                else:
                    wassersteinRatio = 1

            costsPerParamLSx[tuple(paramsLSx.values())] = {'wassersteinRatio': wassersteinRatio}

        #---
        
        costsDf = pd.DataFrame.from_dict(costsPerParamLSx, orient = 'index')
        
        paramsLSxNames = list(paramsLSx.keys())
        costsDf.index.names = paramsLSxNames

        costsDf = costsDf.reset_index(drop = False)
        for paramName, value in paramsEstimator.items():
            costsDf[paramName] = value

        paramNames = paramsLSxNames + list(paramsEstimator.keys())
        costsDf = costsDf.set_index(paramNames)
        
        costsDfList.append(costsDf)
        
    #---
    
    costsDf = pd.concat(costsDfList, axis = 0)
    
    return costsDf

#### Get Wasserstein Distances

In [ ]:
def getWassersteinDistances(densities, yTest, p):
    
    wassersteinDists = list()
    
    for i in range(yTest.shape[0]):
        y = yTest[i]
        values = densities[i][1]
        probs = densities[i][0]

        if len(values.shape) == 1:
            values = values.reshape(-1, 1)
            y = y.reshape(-1, 1)

        wassersteinDists.append(np.sum(probs * np.sum(np.abs(values - y)**p, axis = 1))**(1/p))

    return np.array(wassersteinDists)

In [ ]:
#| hide

# def getWassersteinDistances(densities, yTest, p):
    
#     # CHECK IF TARGET VALUES ARE ONE- OR MULTI-DIMENSIONAL
#     # In the case of one-dimensional arrays, a faster algorithm that
#     # doesn't iterate over the test observations can be used.
#     if len(densities[0][1].shape) == 1:
        
#         lenList = list()
#         valuesList = list()
#         probsList = list()
        
#         for probs, values in densities:
#             lenList.append(values.shape[0])
#             probsList.append(probs)
#             valuesList.append(values)
            
#         lenMax = max(lenList)   
#         insertionCheck = np.arange(lenMax) < np.array(lenList)[:, None]

#         probsArray = np.zeros(shape = (len(probsList), lenMax))
#         valuesArray = np.zeros(shape = (len(valuesList), lenMax))

#         probsArray[insertionCheck] = np.concatenate(probsList, axis = 0)
#         valuesArray[insertionCheck] = np.concatenate(valuesList, axis = 0)
#         yTest = yTest.reshape(-1, 1)

#         wassersteinDists = np.sum(probsArray * np.abs(valuesArray - yTest)**p, axis = 1)**(1/p)
        
#         return wassersteinDists
    
#     #---
    
#     else:
#         wassersteinDists = list()
    
#         for i in range(yTest.shape[0]):
#             y = yTest[i]
#             values = densities[i][1]
#             probs = densities[i][0]

#             if len(values.shape) == 1:
#                 values = values.reshape(-1, 1)
#                 y = y.reshape(-1, 1)

#             wassersteinDists.append(np.sum(probs * np.sum(np.abs(values - y)**p, axis = 1))**(1/p))

#         return np.array(wassersteinDists)

# Unit Tests

# Wasserstein Distance

In [ ]:
# ONE DIMENSIONAL TARGET VALUES
densities = [(np.array([0.4, 0.6]), np.array([2, 4])), 
             (np.array([0.5, 0.5]), np.array([3, 6]))]

y = np.array([3, 4])

wassersteinDists = getWassersteinDistances(densities = densities, 
                                           yTest = y, 
                                           p = 1)

distsByHand = np.array([0.4 * 1 + 0.6 * 1, 0.5 * 1 + 0.5 * 2])

assert np.allclose(wassersteinDists, distsByHand)

#---

# MULTI DIMENSIONAL TARGET VALUES
densities = [(np.array([0.4, 0.6]), np.array([[2, 4], [4, 4]])), 
             (np.array([0.5, 0.5]), np.array([[3, 6], [2, 3]]))]

y = np.array([[3, 4], 
              [4, 2]])

wassersteinDists = getWassersteinDistances(densities = densities, 
                                           yTest = y,
                                           p = 1)

distsByHand = np.array([0.4 * (1 + 0) + 0.6 * (1 + 0), 
                        0.5 * (1 + 4) + 0.5 * (2 + 1)])

assert np.allclose(wassersteinDists, distsByHand)